In [452]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MinMaxScaler
import keras_tuner as kt

In [453]:
TIME_STEPS =4
def create_sequences(X, y, time_steps):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i : i + time_steps])  # Collect time_steps observations
        y_seq.append(y[i + time_steps])  # Predict the next value (shifted target)
    return np.array(X_seq), np.array(y_seq)

In [454]:
df=pd.read_csv("datasets/sensor.csv")
print(df.isnull().sum())

Unnamed: 0             0
timestamp              0
sensor_00          10208
sensor_01            369
sensor_02             19
sensor_03             19
sensor_04             19
sensor_05             19
sensor_06           4798
sensor_07           5451
sensor_08           5107
sensor_09           4595
sensor_10             19
sensor_11             19
sensor_12             19
sensor_13             19
sensor_14             21
sensor_15         220320
sensor_16             31
sensor_17             46
sensor_18             46
sensor_19             16
sensor_20             16
sensor_21             16
sensor_22             41
sensor_23             16
sensor_24             16
sensor_25             36
sensor_26             20
sensor_27             16
sensor_28             16
sensor_29             72
sensor_30            261
sensor_31             16
sensor_32             68
sensor_33             16
sensor_34             16
sensor_35             16
sensor_36             16
sensor_37             16


In [455]:
sensors_to_drop = ['Unnamed: 0', 'timestamp','sensor_15', 'sensor_50']
df = df.drop(columns=sensors_to_drop)

sensor_cols = df.columns[df.isnull().any()].tolist()
df[sensor_cols] = df[sensor_cols].interpolate(method='linear')

# If any remaining NaNs, use forward/backward fill
df[sensor_cols] = df[sensor_cols].fillna(method='ffill')
df[sensor_cols] = df[sensor_cols].fillna(method='bfill')
# Verify no missing values remain
broken_positions = df.index[df["machine_status"] == "BROKEN"].tolist()

# Assign "BROKEN" to 59 previous points
for pos in broken_positions:
    start = max(0, pos - 10)  # Ensure we don't go below index 0
    df.loc[start:pos, "machine_status"] = "BROKEN"

/tmp/ipykernel_203727/2929346405.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[sensor_cols] = df[sensor_cols].fillna(method='ffill')
/tmp/ipykernel_203727/2929346405.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[sensor_cols] = df[sensor_cols].fillna(method='bfill')


In [456]:
y=df["machine_status"]
X=df.drop(columns=['machine_status'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, shuffle=False)
scaler=MinMaxScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

X_train_seq, y_train_seq = create_sequences(X_train, y_train, TIME_STEPS)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, TIME_STEPS)

In [457]:
print(np.shape(df))
unique, counts = np.unique(y, return_counts=True)

# Display results
category_counts = dict(zip(unique, counts))
print(category_counts)


(220320, 51)
{'BROKEN': 77, 'NORMAL': 205766, 'RECOVERING': 14477}


In [458]:
print(y.head())

0    NORMAL
1    NORMAL
2    NORMAL
3    NORMAL
4    NORMAL
Name: machine_status, dtype: object


In [459]:
label_encoder = LabelEncoder()
y_train_seq = label_encoder.fit_transform(y_train_seq)
y_test_seq = label_encoder.transform(y_test_seq)

# Convert to one-hot encoding
num_classes = len(np.unique(y_train_seq))
print(num_classes)
print(np.unique(y_test))
y_train_seq = to_categorical(y_train_seq, num_classes)
y_test_seq = to_categorical(y_test_seq, num_classes)

3
['BROKEN' 'NORMAL' 'RECOVERING']


In [460]:
print(np.unique(y_train_seq))
print(np.argmax(y_train_seq, axis=1))
y_train_classes = np.argmax(y_train_seq, axis=1)

# Compute class weights
class_labels = np.unique(y_train_classes)
class_weights = compute_class_weight(class_weight="balanced", classes=class_labels, y=y_train_classes)

# Convert to a dictionary for Keras
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

[0. 1.]
[1 1 1 ... 1 1 1]


In [461]:

import tensorflow.keras.backend as K
def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, K.floatx())
        cross_entropy = K.binary_crossentropy(y_true, y_pred)
        weight = alpha * y_true + (1 - alpha) * (1 - y_true)
        focal_loss = weight * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.mean(focal_loss)
    return loss

In [462]:
def adaptive_sampler(X, y):
    """ Custom sampler to favor underrepresented classes """
    threshold=500
    rare_classes = [cls for cls in np.unique(y) if np.sum(y == cls) < threshold]
    
    while True:
        indices = []
        for cls in rare_classes:
            indices.extend(np.random.choice(np.where(y == cls)[0], size=10, replace=True))
        indices.extend(np.random.choice(len(y), size=22))  # Add some general samples
        np.random.shuffle(indices)
        yield X[indices], y[indices]

In [463]:

num_features = X_train.shape[1]
inputs = Input(shape=(TIME_STEPS, num_features))
x = layers.Conv1D(64, kernel_size=2, padding="same", activation="relu")(inputs)
filters_list=[192,224,128,160, 160]
for i in range(3):
    shortcut = x  # save input for the skip connection      
    # First convolution in block
    y = layers.Conv1D(filters_list[i], kernel_size=2, padding="same", activation="relu")(x)
    y = layers.BatchNormalization()(y)
    filter2=filters_list[i]
    # Second convolution in block (no activation until after adding the shortcut)
    y = layers.Conv1D(filter2, kernel_size=2, padding="same")(y)
    y = layers.BatchNormalization()(y)

    # If the number of channels does not match, adjust the shortcut
    if shortcut.shape[-1] != filter2:
        shortcut = layers.Conv1D(filter2, kernel_size=1, padding="same")(shortcut)
    # Add the shortcut (residual connection)
    x = layers.Add()([shortcut, y])
    x = layers.Activation("relu")(x)

# Global pooling and output classification layer
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
# Compile the model
model = models.Model(inputs, outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)  # Set LR here
model.compile(optimizer=optimizer, 
            loss='categorical_crossentropy',
            metrics=['accuracy']
            )



In [464]:
from sklearn.metrics import accuracy_score
class PerClassAccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_data, class_names=None):
        """
        Callback to compute per-class accuracy after each epoch.
        
        val_data: (X_val, y_val) validation dataset
        class_names: List of class names for logging
        """
        super().__init__()
        self.X_val, self.y_val = val_data
        self.class_names = class_names if class_names else np.unique(self.y_val)

    def on_epoch_end(self, epoch, logs=None):
        # Get model predictions
        y_pred = self.model.predict(self.X_val, verbose=0)
        y_pred_classes = np.argmax(y_pred, axis=1)  # Convert softmax to class labels
        y_true_classes = np.argmax(self.y_val, axis=1)  # True class labels

        # Compute per-class accuracy
        class_accuracies = {}
        for class_idx in np.unique(y_true_classes):
            mask = (y_true_classes == class_idx)
            acc = accuracy_score(y_true_classes[mask], y_pred_classes[mask])
            class_name = self.class_names[class_idx] if isinstance(self.class_names, list) else class_idx
            class_accuracies[class_name] = acc

        # Log per-class accuracy
        #print(f"\nEpoch {epoch+1} Per-Class Accuracy:")
        #for class_name, acc in class_accuracies.items():
        #    print(f"  - {class_name}: {acc:.4f}")

        logs = logs or {}
        logs.update(class_accuracies)

In [465]:
class_names = ['BROKEN', 'NORMAL', 'RECOVERING']  # Replace with actual class labels

# Add callback to model training
per_class_acc_callback = PerClassAccuracyCallback((X_test_seq, y_test_seq), class_names)
history =model.fit(X_train_seq, y_train_seq, epochs=20, validation_data=(X_test_seq, y_test_seq), batch_size=128, callbacks=[per_class_acc_callback])

Epoch 1/20
489/517 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9464 - loss: 0.1423

2025-02-20 16:33:46.597804: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2033', 12 bytes spill stores, 12 bytes spill loads



517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9487 - loss: 0.1363

2025-02-20 16:33:49.456415: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_263', 12 bytes spill stores, 12 bytes spill loads



517/517 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.9488 - loss: 0.1361 - val_accuracy: 0.9845 - val_loss: 0.0526 - BROKEN: 0.0000e+00 - NORMAL: 0.9976 - RECOVERING: 0.8097
Epoch 2/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9991 - loss: 0.0033 - val_accuracy: 0.9382 - val_loss: 0.1256 - BROKEN: 0.0182 - NORMAL: 0.9344 - RECOVERING: 0.9952
Epoch 3/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9995 - loss: 0.0014 - val_accuracy: 0.9848 - val_loss: 0.0473 - BROKEN: 0.0182 - NORMAL: 0.9864 - RECOVERING: 0.9674
Epoch 4/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9994 - loss: 0.0015 - val_accuracy: 0.9828 - val_loss: 0.0621 - BROKEN: 0.0727 - NORMAL: 0.9823 - RECOVERING: 0.9954
Epoch 5/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9995 - loss: 0.0015 - val_accuracy: 0.9693 - val_loss: 0.0924 - BROKEN: 0.0727 - NORMAL: 0.9677 - RECOVERING: 0.9963
Epoch 6/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9997 - loss: 0.0013

In [466]:
# Evaluate the model
#loss, metric = model.evaluate(X_test_seq, y_test_seq)
#print(f"Test Loss: {loss:.4f}, Test {'Accuracy' if 'num_classes' in locals() else 'MAE'}: {metric:.4f}")

In [467]:
#import seaborn as sns
#from sklearn.metrics import confusion_matrix
#y_pred=model.predict(X_test_seq)
#y_hat = np.argmax(y_pred, axis=1)  # Get the highest probability class index
#y_true = np.argmax(y_test_seq, axis=1)  # Ensure y_true is also class indices
#print(y_true)
#print(y_hat)

#cm = confusion_matrix(y_true, y_hat)

# Plot using seaborn
#plt.figure(figsize=(8, 6))
#sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(num_classes), yticklabels=range(num_classes))
#plt.xlabel("Predicted Label")
#plt.ylabel("True Label")
#plt.title("Confusion Matrix")
#plt.gca().invert_yaxis()
#plt.show()

In [468]:
#Good start: 3 timesteps, 30 minute broken window. 99% acc, 4 (0,0)
#Good start: 2 timesteps, 30 minute broken window. 98% acc, 5 (0,0)
